# 08 - RandomForest

Realizando a predição com DecisionTree. Lembrando que a acurácia para os dados de treino precisa ser superior a 78% para que seja considerada válida para testarmos com os dados de teste e que, nos dados de teste, precisa ser superior a **76.5%** para ser considerada superior a `DecisionTree`.

## Preparando o ambiente

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, GroupKFold, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler

## Carregando os dados